In [6]:
!pip install html5lib

     -------------------------------------- 112.2/112.2 kB 3.3 MB/s eta 0:00:00


In [11]:
import html5lib
print(html5lib.__version__)


1.1


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# La Liga standings URL
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

# Function to scrape match and shooting data for a given team URL
def scrape_team_data(team_url):
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text, 'html5lib')  # Specify the parser explicitly
    
    # Find the table containing scores & fixtures
    matches = pd.read_html(data.text, match="Scores & Fixtures", flavor='html5lib')[0]
    
    # Find the link for shooting statistics
    links = soup.find_all('a', href=True)
    shooting_link = next((l['href'] for l in links if 'all_comps/shooting/' in l['href']), None)
    
    if shooting_link:
        # Fetch shooting statistics
        shooting_url = f"https://fbref.com{shooting_link}"
        data_shooting = requests.get(shooting_url)
        shooting = pd.read_html(data_shooting.text, match="Shooting", flavor='html5lib')[0]
        shooting.columns = shooting.columns.droplevel()
        
        # Merge matches and shooting data on 'Date'
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        return team_data
    else:
        return pd.DataFrame()

# Main scraping loop
data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html5lib')  # Specify the parser explicitly

# Find the table containing team URLs
standings_table = soup.select('table.stats_table')[0]

# Extract links from the table
links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]

# Form complete URLs
team_urls = [f"https://fbref.com{l}" for l in links]

all_matches = []

# Loop through each team URL
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    team_data = scrape_team_data(team_url)
    
    if not team_data.empty:
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)  # Adding a delay to not overwhelm the server

# Concatenate all dataframes into one
match_df = pd.concat(all_matches, ignore_index=True)

# Define the file path
file_path = "C:/University work/la_liga_matches.csv"

# Save to CSV
match_df.to_csv(file_path, index=False)

print(f"Scraping and data processing completed. Data saved to '{file_path}'.")



Scraping and data processing completed. Data saved to 'C:/University work/la_liga_matches.csv'.


In [5]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Team
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2,0,Athletic Club,...,Jesús Gil,Match Report,NaN,14.0,8.0,16.0,2.0,0,0,Real Madrid
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3,1,Almería,...,José Sánchez,Match Report,NaN,25.0,9.0,17.0,1.0,0,0,Real Madrid
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1,0,Celta Vigo,...,Isidro Díaz de Mera,Match Report,NaN,9.0,2.0,19.4,0.0,0,1,Real Madrid
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2,1,Getafe,...,Mario Melero,Match Report,NaN,26.0,12.0,17.7,0.0,0,0,Real Madrid
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2,1,Real Sociedad,...,César Soto,Match Report,NaN,17.0,8.0,15.9,1.0,0,0,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,2024-05-05,21:00,La Liga,Matchweek 34,Sun,Away,L,0,3,Sevilla,...,Mateo Busquets,Match Report,NaN,8.0,3.0,26.3,0.0,0,0,Granada
907,2024-05-11,18:30,La Liga,Matchweek 35,Sat,Home,L,0,4,Real Madrid,...,Pablo González,Match Report,NaN,12.0,1.0,13.6,0.0,0,0,Granada
908,2024-05-15,19:30,La Liga,Matchweek 36,Wed,Away,L,1,2,Rayo Vallecano,...,Isidro Díaz de Mera,Match Report,NaN,10.0,1.0,18.2,1.0,0,0,Granada
909,2024-05-19,19:00,La Liga,Matchweek 37,Sun,Home,L,1,2,Celta Vigo,...,Javier Alberola,Match Report,NaN,10.0,3.0,19.8,0.0,0,1,Granada


In [8]:
# Define the file path with double backslashes
file_path = "C:\\University work\\la_liga_matches.csv"
match_df.to_csv("la_liga_matches.csv", index=False)

print(f"Scraping and data processing completed. Data saved to 'la_liga_matches.csv'.")


Scraping and data processing completed. Data saved to 'la_liga_matches.csv'.
